Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI 101 tutorial

In this tutorial, we introduce the basic usage of MONAI to solve a classification problem with the MedNIST dataset.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/2d_classification/mednist_101.ipynb)

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"

## Setup imports

In [5]:
import numpy as np
import os
from pathlib import Path
import tempfile
import torch

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer
from monai.inferers import SimpleInferer
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, ToTensorD, Compose
from monai.utils import set_determinism

print_config()

MONAI version: 1.2.0rc2+45.gb1161b00
Numpy version: 1.22.2
Pytorch version: 1.14.0a0+410ce96
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: b1161b0099ed4b5617525227f9e2a1059fdcdb63
MONAI __file__: /workspace/monai/monai-in-dev/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.0.1
scikit-image version: 0.20.0
Pillow version: 9.2.0
Tensorboard version: 2.9.0
gdown version: 4.6.4
TorchVision version: 0.15.0a0
tqdm version: 4.64.1
lmdb version: 1.4.0
psutil version: 5.9.4
pandas version: 1.5.2
einops version: 0.6.0
transformers version: 4.21.3
mlflow version: 2.2.2
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [6]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/data


## Use MONAI transforms to preprocess data

Medical images require specialized methods for I/O, preprocessing, and augmentation.
They often follow specific formats, are handled with specific protocols, and the data arrays are often high-dimensional.

In this example, we will perform image loading, data format verification, intensity scaling, and conversion to Tensor with four `monai.transforms` listed below, and compose a pipeline ready to be used in next steps.

In [7]:
transform = Compose(
    [
        LoadImageD(keys="image", image_only=True),
        EnsureChannelFirstD(keys="image"),
        ScaleIntensityD(keys="image"),
        ToTensorD(keys=["image", "label"]),
    ]
)

## Prepare datasets using MONAI Apps

We use `MedNISTDataset` in MONAI Apps to download a dataset to the specified directory and perform the pre-processing steps in the `monai.transforms` compose.

The MedNIST dataset was gathered from several sets from [TCIA](https://wiki.cancerimagingarchive.net/display/Public/Data+Usage+Policies+and+Restrictions),
[the RSNA Bone Age Challenge](http://rsnachallenges.cloudapp.net/competitions/4),
and [the NIH Chest X-ray dataset](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

The dataset is kindly made available by [Dr. Bradley J. Erickson M.D., Ph.D.](https://www.mayo.edu/research/labs/radiology-informatics/overview) (Department of Radiology, Mayo Clinic)
under the Creative Commons [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/).

If you use the MedNIST dataset, please acknowledge the source. 

In [8]:
dataset = MedNISTDataset(root_dir=root_dir, transform=transform, section="training", download=True)

2023-04-12 13:37:13,587 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-04-12 13:37:13,587 - INFO - File exists: /workspace/data/MedNIST.tar.gz, skipped downloading.
2023-04-12 13:37:13,588 - INFO - Non-empty folder exists in /workspace/data/MedNIST, skipped extracting.


Loading dataset: 100%|██████████| 47164/47164 [00:24<00:00, 1938.94it/s]


## Define a network and a supervised trainer

To train a model that can perform the classification task, we will use the DenseNet-121 which is known for its performance on the ImageNet dataset.

For a typical supervised training workflow, MONAI provides `SupervisedTrainer` to define the hyper-parameters.

In [9]:
max_epochs = 5
net = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=net,
    optimizer=torch.optim.Adam(net.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
)

## Run the training

In [10]:
trainer.run()

## Check the prediction on the test dataset

In [29]:
dataset_dir = Path(root_dir, "MedNIST")
class_names = sorted(f"{x.name}" for x in dataset_dir.iterdir() if x.is_dir())
testdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="test", download=False, runtime_cache=True)

max_items_to_print = 10
print(f"Classes: {class_names}")
for item in DataLoader(testdata, batch_size=1, num_workers=0):
    prob = np.array(net(item["image"].to("cuda:0")).detach().to("cpu"))[0]
    gt = item["class_name"]
    print(f"Class prediction probablity is {prob}. Ground-truth: {gt}")
    max_items_to_print -= 1
    if max_items_to_print == 0:
        break

Classes: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Class prediction probablity is [-0.1402147  -0.13285248 -0.5135751   0.03817053  0.11568966  0.09673146]. Ground-truth: ['AbdomenCT']
Class prediction probablity is [-0.13782811 -0.16204707 -0.5100275   0.02717766  0.0833546   0.07674835]. Ground-truth: ['BreastMRI']
Class prediction probablity is [-0.12441307 -0.17982616 -0.5009079   0.00481447  0.08921886  0.0980939 ]. Ground-truth: ['ChestCT']
Class prediction probablity is [-0.13865665 -0.13482976 -0.528063   -0.04141937  0.08532636  0.07730866]. Ground-truth: ['CXR']
Class prediction probablity is [-0.11238856 -0.13422833 -0.49332258 -0.00906343  0.09495345  0.07716608]. Ground-truth: ['Hand']
Class prediction probablity is [-0.09088446 -0.16400021 -0.52175134  0.01733851  0.06469806  0.05816339]. Ground-truth: ['HeadCT']
Class prediction probablity is [-0.14555132 -0.1543928  -0.52901495 -0.01370656  0.06382443  0.08063938]. Ground-truth: ['HeadCT']
Class pre